<a href="https://colab.research.google.com/github/Prkhar05/Tiny_Text2SQL/blob/main/sql_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset,concatenate_datasets,Dataset,DatasetDict
import pandas as pd

In [ ]:
dataset_name1="b-mc2/sql-create-context"
dataset_name2="Clinton/Text-to-sql-v1"
dataset_name3="zerolink/zsql-sqlite-dpo"
dataset_name4="spider"
dataset_name5="PipableAI/pip-txt-to-sql-spider-bird-dataset"
dataset_name6="gretelai/synthetic_text_to_sql"
dataset_name7='NumbersStation/NSText2SQL'

In [ ]:
def loading(dataset_name):
  d = load_dataset(dataset_name)
  dataset=d[list(d.keys())[0]]
  keys=list(d.keys())
  keys.pop(0)
  for key in keys:
    dataset=concatenate_datasets([dataset,d[key]])
  print(f"Some info of {dataset_name}")
  print(dataset.shape)
  print(dataset.column_names)
  print(dataset[0])
  return dataset
def prepare_dataset(dataset,rename_columns,remove_columns):
  # dataset = load_dataset(dataset_name)
  # dataset=dataset['train']
  for column in rename_columns:
    dataset = dataset.rename_column(column[0],column[1])
  dataset = dataset.remove_columns(remove_columns)
  return dataset

In [ ]:
rawdataset1=loading(dataset_name1)
rawdataset2=loading(dataset_name2)
rawdataset3=loading(dataset_name3)
rawdataset5=loading(dataset_name5)
rawdataset6=loading(dataset_name6)

In [ ]:
dataset1=prepare_dataset(rawdataset1,[['context','schema'],['answer','query']],[])
dataset2=prepare_dataset(rawdataset2,[['input','schema'],['response','query'],['instruction','question']],['source', 'text'])
dataset3=prepare_dataset(rawdataset3,[['rejected','query']],['chosen', 'weight'])
dataset5=prepare_dataset(rawdataset5,[],[])
dataset6=prepare_dataset(rawdataset6,[['sql_context','schema'],['sql','query'],['sql_prompt','question']],['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_explanation'])

In [ ]:
dataset7=load_dataset(dataset_name7)
dataset7=dataset7['train']
dataset7=pd.DataFrame(dataset7)
splt="-- Using valid SQLite, answer the following questions for the tables provided above.\n\n--"
schema=[]
question=[]
for data in dataset7['instruction']:
  d=data.split(splt)
  schema.append(d[0])
  question.append(d[1])
data = {'schema': schema, 'question': question, 'query': dataset7['output']}
dataset7 = pd.DataFrame(data, columns=['schema', 'question', 'query'])
dataset7=dataset7.drop(2)

In [ ]:
dataset=pd.concat([pd.DataFrame(dataset1),pd.DataFrame(dataset2),pd.DataFrame(dataset3),pd.DataFrame(dataset5),pd.DataFrame(dataset6),dataset7]).drop_duplicates(subset=['question'])
dataset = Dataset.from_pandas(dataset)
dataset

In [ ]:
dataset.push_to_hub("Cynaptics/Test2Sql")

# splitting dataset

In [ ]:
dataset=load_dataset("Cynaptics/Test2Sql")
dataset=dataset['train']
d=pd.DataFrame(dataset)
d=d.drop(columns=["__index_level_0__"])

In [ ]:
import random
random_numbers_1000 = random.sample(range(1, 489956), 1000)
random_numbers_10000 = random.sample(range(1, 489956), 10000)

In [ ]:
small_train=d.iloc[random_numbers_10000]
small_validation=d.iloc[random_numbers_1000]

In [ ]:
dataset=DatasetDict({
    "train":dataset,
    "small_train":Dataset.from_pandas(small_train),
    "small_validation":Dataset.from_pandas(small_validation)
})
dataset

In [ ]:
dataset.push_to_hub("Cynaptics/Test2Sql")

#conversion

In [ ]:
dataset=load_dataset("Cynaptics/Test2Sql")
dataset

In [ ]:
def prompt_pip_sql(schema,question,query):
  prompt = f"""<schema>{schema}</schema>
    <question>{question}</question>
    <sql>{query}</sql>"""
  return prompt

In [ ]:
dataset['small_validation']['schema'][0]

'CREATE TABLE table_name_28 (date VARCHAR, week VARCHAR)'

In [ ]:
import tqdm
dicti={}
for key in tqdm.tqdm(dataset.keys()):
  dicti[key]={}
  dicti[key]['prompt'] =[prompt_pip_sql(schema,question,query) for (schema,question,query) in zip(dataset[key]['schema'],dataset[key]['question'],dataset[key]['query'])]
  dicti[key]=Dataset.from_dict(dicti[key])

100%|██████████| 5/5 [00:10<00:00,  2.18s/it]


In [ ]:
dicti=DatasetDict(dicti)
dicti

In [ ]:
dicti.push_to_hub("Cynaptics/Test2Sql_InstructionTuned")

#overview

In [ ]:
# num=0
# for data in dataset1:
#   schema=data['schema']
#   filtered=dataset2.filter(lambda d: d['schema']==schema)
#   num+=filtered.shape[0]
# num

In [ ]:
# concat=concatenate_datasets([dataset1,dataset2,dataset3,dataset5])
# d=pd.DataFrame(concat)
# # # d=d[['question','schema']]
# duplics=d.duplicated(subset=['question'])
# # duplics=d.duplicated()
# duplics.sum()

In [ ]:
# d[duplics]

In [ ]:
# def func(data):
#   if(data['question']=="What are the lowest points with 2013 as the year, and goals less than 0?"):
#     print(data)
# dataset.filter(func)

In [ ]:
# duplics=d[d['query'].str.len()>65].duplicated(subset=['question'])
# d[d['query'].str.len()>65][duplics]

In [ ]:
# group=d[duplics].groupby('schema')
# group.sum()
# group.count()

In [ ]:
# queries=pd.DataFrame(dataset['query'],columns=['query'])
# queries['length']=queries['query'].apply(len)

In [ ]:
# d.sort_values(by='length',ascending=False)

In [ ]:
# queries.sort_values(by='lenght', ascending=False)
# queries.describe()
# 174640=>(...,300)
# 16532=>(300,500)
# 9414=>(500,1000)
# 1238=>(1000,2000)
# 70=>(2000,3000)
# 12=>(3000,...)

# print(d[(d['length'] <= 300)].shape[0])
# print(d[(d['length'] >= 300) & (d['length'] <= 500)].shape[0])
# print(d[(d['length'] >= 500) & (d['length'] <= 1000)].shape[0])
# print(d[(d['length'] >= 1000) & (d['length'] <= 2000)].shape[0])
# print(d[(d['length'] >= 2000) & (d['length'] <= 3000)].shape[0])
# print(d[(d['length'] >= 3000)].shape[0])

In [ ]:
# where=0
# group=0
# order=0
# join=0
# having=0
# distinct=0
# limit=0
# union=0
# offset=0
# like=0
# _in=0
# between=0
# _case=0
# exists=0

# def func(query):
#   global where, group, order, join, having, distinct, limit, union, offset,like, _in, _case, between, exists
#   query=query.split(' ')
#   if "WHERE" in query:
#     where=where+1
#   if "GROUP" in query:
#     group=group+1
#   if "ORDER" in query:
#     order=order+1
#   if "JOIN" in query:
#     join=join+1
#   if "HAVING" in query:
#     having=having+1
#   if "DISTINCT" in query:
#     distinct=distinct+1
#   if "LIMIT" in query:
#     limit=limit+1
#   if "UNION" in query:
#     union=union+1
#   if "OFFSET" in query:
#     offset=offset+1
#   if "LIKE" in query:
#     like=like+1
#   if "IN" in query:
#     _in=_in+1
#   if "BETWEEN" in query:
#     between=between+1
#   if "CASE" in query:
#     _case=_case+1
#   if "EXISTS" in query:
#     exists=exists+1

# d['query'].map(func)
# print(f"where {where},group {group},order {order},join {join},having {having},distinct {distinct},limit {limit},union {union},offset {offset},like {like},between {between},exists {exists},case {_case},in {_in}")

# # where 176657,group 27972,order 36733,join 39510,having 1800,distinct 11548,limit 15878,union 309,offset 1052,like 8189,between 5119,exists 0,case 396,in 18801

# spider

In [ ]:
import gdown

In [ ]:
# https://drive.google.com/file/d/1mkCx2GOFIqNesD4y8TDAO1yX1QZORP5w/view?usp=sharing  #test suite dataset
# https://drive.usercontent.google.com/open?id=1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m&authuser=0 #spider dataset
# https://drive.usercontent.google.com/open?id=1Y3ydpFiQQ3FC0bzdfy3groV95O_f1nXF&authuser=0 #cosql dataset
# https://drive.usercontent.google.com/open?id=1Uu7NMHTR1tdQw1t7bAuM7OPU4LElVKfg&authuser=0 #sparc dataset
spider_id="1iRDVHLr4mX2wQKSgA9J8Pire73Jahh0m"
spider_file="spiderdatasets"

cosql_id="1Y3ydpFiQQ3FC0bzdfy3groV95O_f1nXF"
cosql_file="cosqldatasets"

sparc_id="1Uu7NMHTR1tdQw1t7bAuM7OPU4LElVKfg"
sparc_file="sparcdatasets"

In [ ]:
# !unzip -q spiderdatabases.zip -d spiderdatabases/

In [ ]:
import zipfile
import json
def download_zip(file_id,output_file):
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file, quiet=False)
def extract_zip(file_name) :
    with zipfile.ZipFile(f"/content/{file_name}.zip", 'r') as zip_ref:
        zip_ref.extractall(f"{file_name}/")
def _zip(file_id,output_file):
    download_zip(file_id,f'{output_file}.zip')
    extract_zip(output_file)

In [ ]:
_zip(spider_id,spider_file)
_zip(cosql_id,cosql_file)
_zip(sparc_id,sparc_file)

In [ ]:
dataset=load_dataset("Cynaptics/Test2Sql")
dataset=dataset['train']

In [ ]:
with open("/content/sparcdatasets/sparc/dev.json", 'r') as f:
    data = f.read()
    data=json.loads(data)

In [ ]:
def func(data):
  if(data['question']=="What is the start date of each appointment?"):
    print(data)
dataset.filter(func)

In [ ]:
len(data),type(data)

(422, list)

In [ ]:
data[1]

In [ ]:
l=[]
for d in data:
  l.append(d['question'])
len(l)

2147

In [ ]:
pd.concat([pd.DataFrame(dataset['question']),pd.DataFrame(l)]).duplicated().sum()

30

In [ ]:
# list(data.keys())[1]
# for key in data['24Egz2cFCmN77SbNs'].keys():
#   print(key,data['24Egz2cFCmN77SbNs'][key])

In [ ]:
with open("/content/spiderdatabase/spider/database/academic/schema.sql", 'r') as sql_file:
    sql_commands = sql_file.read()
    print(sql_commands)

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('/content/database/database/academic/academicv1223round10group0.sqlite')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all rows from the query result
tables = cursor.fetchall()

# Print the retrieved table names
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()
conn.close()


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('/content/database/database/academic/academicv1223round10group3.sqlite')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Example: Retrieve and print the contents of a specific table
table_name = 'author'  # Replace 'your_table_name' with the actual table name
# cursor.execute("INSERT INTO author (aid, homepage, name, oid) VALUES (2, 'http://www.mm.com', 'John', 1002);")
cursor.execute(f"SELECT * FROM {table_name}")

# Fetch all rows from the query result
rows = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print("Column names:", column_names)

# Print the retrieved rows
for row in rows:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()
